## Determine Main Contributers to Electricity RRP

## Stage 3: Combine Generator and Interconnector Data with State Price and Demand Data

In [1]:
# Import libraries 
# I've included a handful of libraries here that might be useful
import glob
import os, zipfile
import pathlib
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import sys
import time
from IPython.display import display # Used to display multiple pandas tables in one cell
from datetime import datetime, time
from datetime import timedelta
from scipy import stats
from pprint import pprint

In [ ]:
# Interconnector 30min Average Data for required Month obtained from Stage 1
interconn_file ='F:/RMIT_MasterofAnalytics/LEP_DataScience/Assignments/Datasets/DispatchIS_Feb17_30minAverage.csv'
df_interconn = pd.read_csv(interconn_file,sep=',',skiprows=0,error_bad_lines=False,warn_bad_lines=False)

In [3]:
# Generator 30min Average Data for required Month obtained from Stage 2
scada_file ='F:/RMIT_MasterofAnalytics/LEP_DataScience/Assignments/Datasets/DispatchScada_Feb17_30minAverage.csv'
df_scada = pd.read_csv(scada_file,sep=',',skiprows=0,error_bad_lines=False,warn_bad_lines=False)

State Price and Demand Data for required Month and required State obtained from:
https://www.aemo.com.au/Electricity/National-Electricity-Market-NEM/Data-dashboard#aggregated-data

In [5]:
priceDemand_file ='F:/RMIT_MasterofAnalytics/LEP_DataScience/Assignments/Datasets/PriceandDemand/PRICE_AND_DEMAND_201702_NSW1.csv'
df_priceDemand = pd.read_csv(priceDemand_file,sep=',',skiprows=0,error_bad_lines=False,warn_bad_lines=False)

Generator Information obtained fro:Australian Renewable Energy Mapping Infrastructure: https://nationalmap.gov.au/renewables/ 

In [6]:
NSW1_file ='F:/RMIT_MasterofAnalytics/LEP_DataScience/Assignments/NSW1_generators.csv'
df_NSW1 = pd.read_csv(NSW1_file,sep=',',skiprows=0,error_bad_lines=False,warn_bad_lines=False)

In [7]:
print(df_NSW1.head(5))

   AGL      Duid Fuel Source - Descriptor  \
0  NOT  TARALGA1                     Wind   
1  NOT  WOODLWN1                     Wind   
2  NOT  GUNNING1                     Wind   
3  NOT  CULLRGWF                     Wind   
4  NOT  CAPTL_WF                     Wind   

                                         Participant Unit Size (MW)  \
0  Taralga Wind Farm Nominees No 2 Pty Ltd (ATF T...          106.8   
1                              Woodlawn Wind Pty Ltd            2.1   
2           Gunning Wind Energy Developments Pty Ltd            2.1   
3                  Origin Energy Electricity Limited              2   
4                 Infigen Energy Markets Pty Limited              2   

         Lat         Lon  
0 -34.427076  149.857640  
1 -35.086060  149.571606  
2 -34.695831  149.383175  
3 -34.804982  149.400130  
4 -35.166031  149.518569  


In [8]:
# Add the interconnector Names to the Generator Information
NSWGenList = list(df_NSW1['Duid'])
NSWGenList = NSWGenList + ['N-Q-MNSP1','NSW1-QLD1','VIC1-NSW1']

In [9]:
print(NSWGenList)

['TARALGA1', 'WOODLWN1', 'GUNNING1', 'CULLRGWF', 'CAPTL_WF', 'BOCORWF1', 'GULLRWF1', 'TUMUT3', 'MURRAY', 'UPPTUMUT', 'BLOWERNG', 'SHGEN', 'GUTHEGA', 'WYANGALA', 'JOUNAMA1', 'KEEPIT', 'BURRIN', 'WYANGALB', 'THEDROP1', 'JNDABNE1', 'BROWNMT', 'COPTNHYD', 'BDONGHYD', 'PINDARI', 'GLBWNHYD', 'MOREESF1', 'MLSP', 'ROYALLA1', 'CESF1', 'NYNGAN1', 'BROKENH1', 'TALWA1', 'CG4', 'CG3', 'CG2', 'CG1', 'URANQ14', 'URANQ13', 'URANQ12', 'URANQ11', 'HVGTS', 'ERGT01', 'GB01', 'STGEORG1', 'NINEWIL1', 'WESTCBT1', 'WESTILL1', 'BANKSPT1', 'WILGB01', 'WILGAPK', 'TERALBA', 'ER04', 'ER03', 'ER02', 'ER01', 'MP2', 'MP1', 'VP6', 'VP5', 'BW04', 'BW03', 'BW02', 'BW01', 'LD04', 'LD03', 'LD02', 'LD01', 'CONDONG1', 'BWTR1', 'N-Q-MNSP1', 'NSW1-QLD1', 'VIC1-NSW1']


In [10]:
print(df_interconn.head(5))

   N-Q-MNSP1   NSW1-QLD1   T-V-MNSP1   V-S-MNSP1        V-SA   VIC1-NSW1  \
0  -9.000000  199.256178 -474.199274  -69.953338  308.379914  771.586480   
1 -51.116667 -328.242177 -473.773893   -4.744007  202.867875  651.377975   
2 -11.000000   65.161715 -478.000000  -20.358867  167.735778  896.018287   
3   9.633333  241.221697 -478.000000 -110.775287  369.339198  913.923222   
4  -6.000000  231.446037 -478.000000 -123.852095  367.907317  958.787083   

              DATETIME  
0  2017-02-01 00:30:00  
1  2017-02-01 01:00:00  
2  2017-02-01 01:30:00  
3  2017-02-01 02:00:00  
4  2017-02-01 02:30:00  


In [11]:
##Need to change Polarity of Interconnectors
#N-Q-MNSP1 provides electricity to Qld from NSW so should change sign to be default of negative
#NSW1-QLD1 provides electricity to Qld from NSW so should change sign to be default of negative
#VIC1-NSW1 provides electricity to NSW from VIC so should be left as it is
df_interconn['N-Q-MNSP1'] = df_interconn['N-Q-MNSP1']* -1
df_interconn['NSW1-QLD1'] = df_interconn['NSW1-QLD1']* -1
df_interconn['VIC1-NSW1'] = df_interconn['VIC1-NSW1']

In [12]:
print(df_interconn.head(5))

   N-Q-MNSP1   NSW1-QLD1   T-V-MNSP1   V-S-MNSP1        V-SA   VIC1-NSW1  \
0   9.000000 -199.256178 -474.199274  -69.953338  308.379914  771.586480   
1  51.116667  328.242177 -473.773893   -4.744007  202.867875  651.377975   
2  11.000000  -65.161715 -478.000000  -20.358867  167.735778  896.018287   
3  -9.633333 -241.221697 -478.000000 -110.775287  369.339198  913.923222   
4   6.000000 -231.446037 -478.000000 -123.852095  367.907317  958.787083   

              DATETIME  
0  2017-02-01 00:30:00  
1  2017-02-01 01:00:00  
2  2017-02-01 01:30:00  
3  2017-02-01 02:00:00  
4  2017-02-01 02:30:00  


In [13]:
#Remove unnecessary Interconnectors
df_interconn.drop(['V-SA','V-S-MNSP1','T-V-MNSP1'], axis=1,inplace=True)

### Combine Interconnector Information with Generator Information

In [15]:
df_interconn['DATETIME'] = pd.to_datetime(df_interconn['DATETIME'])
df_scada['DATETIME'] = pd.to_datetime(df_scada['DATETIME'])

In [16]:
print(df_interconn['DATETIME'].describe())
print(df_scada['DATETIME'].describe())

count                    1345
unique                   1345
top       2017-02-11 09:00:00
freq                        1
first     2017-02-01 00:30:00
last      2017-03-01 00:30:00
Name: DATETIME, dtype: object
count                    1345
unique                   1345
top       2017-02-11 09:00:00
freq                        1
first     2017-02-01 00:30:00
last      2017-03-01 00:30:00
Name: DATETIME, dtype: object


In [17]:
df_scadaIS = pd.merge(df_scada, df_interconn, on='DATETIME')

In [ ]:
df_scadaIS=df_scadaIS.rename(columns = {'DATETIME':'SETTLEMENTDATE'})

### Combine Generator & Interconnector Data with Price and Demand Data

In [21]:
df_priceDemand.drop(['PERIODTYPE'], axis=1,inplace=True)

In [23]:
# Generator & Interconnector Data has one too many time periods - so remove this
df_scadaIS = df_scadaIS[df_scadaIS['SETTLEMENTDATE']!='2017-03-01 00:30:00']

In [25]:
print(df_priceDemand.shape)
df_priceDemand['SETTLEMENTDATE'] = pd.to_datetime(df_priceDemand['SETTLEMENTDATE'], format='%Y/%m/%d %H:%M:%S', errors='coerce')

(1344, 4)


In [27]:
# Make sure both datasets are the same length
print(df_scadaIS.shape)
print(df_priceDemand.shape)

(1344, 276)
(1344, 4)


In [28]:
# Make sure both datasets have the same timestamp since we will merge on this
print(df_priceDemand['SETTLEMENTDATE'].describe())
print(df_scadaIS['SETTLEMENTDATE'].describe())

count                    1344
unique                   1344
top       2017-02-11 09:00:00
freq                        1
first     2017-02-01 00:30:00
last      2017-03-01 00:00:00
Name: SETTLEMENTDATE, dtype: object
count                    1344
unique                   1344
top       2017-02-11 09:00:00
freq                        1
first     2017-02-01 00:30:00
last      2017-03-01 00:00:00
Name: SETTLEMENTDATE, dtype: object


In [29]:
# Merge Datasets into Combined dataset
df_combined = pd.merge(df_scadaIS, df_priceDemand, on='SETTLEMENTDATE')

In [30]:
print(df_combined.head(5))
print(df_combined.shape)

   AGLHAL  AGLSOM  ANGAST1      ARWF1  BALDHWF1  BARCALDN  BARCSF1  BARRON-1  \
0     0.0     0.0      0.0  60.720000  1.901800       0.0      0.1       0.0   
1     0.0     0.0      0.0  40.400000  0.257333       0.0      0.1       0.0   
2     0.0     0.0      0.0  35.500000  0.000000       0.0      0.1       0.0   
3     0.0     0.0      0.0  29.883333  0.000000       0.0      0.1       0.0   
4     0.0     0.0      0.0  16.683333  0.006667       0.0      0.1       0.0   

   BARRON-2  BASTYAN  ...         YWPS2       YWPS3       YWPS4  \
0       0.0      0.0  ...    374.134680  349.057216  371.187848   
1       0.0      0.0  ...    358.770888  336.429957  355.718990   
2       0.0      0.0  ...    368.378478  351.014355  372.890932   
3       0.0      0.0  ...    374.560765  349.838185  374.443458   
4       0.0      0.0  ...    373.516455  352.943248  374.067087   

       SETTLEMENTDATE  N-Q-MNSP1   NSW1-QLD1   VIC1-NSW1  REGION  TOTALDEMAND  \
0 2017-02-01 00:30:00   9.000000 -1

In [31]:
## Filter on NSW1 Generators
RequiredCols = NSWGenList + ['SETTLEMENTDATE','REGION','TOTALDEMAND','RRP']
colstodrop = [c for c in df_combined.columns.tolist() if c not in RequiredCols]
df_NSW_Feb17_combined = df_combined.drop(colstodrop, axis=1)

In [32]:
print(df_NSW_Feb17_combined.shape)

(1344, 52)


In [33]:
print(df_NSW_Feb17_combined.head(5))

    BLOWERNG  BOCORWF1  BROKENH1        BW01        BW02        BW03  \
0  67.439002  0.580000       0.0  479.845020  481.540012  481.149978   
1  67.435752  0.675000       0.0  446.108340  429.449992  425.895835   
2  67.464288  0.368333       0.0  428.825010  435.195823  425.820822   
3  67.450837  0.131667       0.0  447.845825  444.558330  447.683330   
4  67.435000  0.000000       0.0  430.450052  432.241660  439.516645   

         BW04   CAPTL_WF  CG1  CG2  ...           VP5         VP6   WOODLWN1  \
0  489.030004  27.695203  0.0  0.0  ...    450.919996  452.055004  15.798596   
1  437.433350  24.160001  0.0  0.0  ...    395.958330  390.312500  11.245500   
2  444.191652  38.949333  0.0  0.0  ...    377.545840  374.070832  17.487003   
3  461.845827  50.253333  0.0  0.0  ...    472.245818  474.241668  17.964000   
4  447.474978  32.939332  0.0  0.0  ...    494.541670  492.216670  13.578008   

       SETTLEMENTDATE  N-Q-MNSP1   NSW1-QLD1   VIC1-NSW1  REGION  TOTALDEMAND  \
0 201

In [34]:
outputfile='F:/RMIT_MasterofAnalytics/LEP_DataScience/Assignments/Datasets/NSW1_Feb17_DispatchData_Interconnectors.csv'
df_NSW_Feb17_combined.to_csv(outputfile,index=False)